In [59]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

In [60]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-04-28 09:07:31,0.0,-0.058448,2.370000e-09,1.050000e-09,4.996000e-09,6.208000e-07,1.0754,2.0,11.111,252.524,0.000025,0.000002,NaN,NaN
1,NaN,2023-04-28 09:07:39,8.0,-0.058523,2.350000e-09,1.050000e-09,4.996000e-09,6.216000e-07,1.0754,2.0,10.999,252.412,0.000036,0.000002,NaN,NaN
2,NaN,2023-04-28 09:07:48,17.0,-0.058374,2.380000e-09,1.050000e-09,4.996000e-09,6.224000e-07,1.0754,2.0,11.111,252.300,0.000037,0.000002,NaN,NaN
3,NaN,2023-04-28 09:07:56,25.0,-0.058055,2.380000e-09,1.040000e-09,4.996000e-09,6.231000e-07,1.0754,2.0,10.999,252.412,0.000035,0.000002,NaN,NaN
4,NaN,2023-04-28 09:08:05,34.0,-0.058201,2.360000e-09,1.050000e-09,4.996000e-09,6.238000e-07,1.0754,2.0,10.887,252.412,0.000023,0.000002,NaN,NaN


In [61]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = vaclog["1mbar inj"].values
x = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.10*x+0.39


In [62]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot((vaclog["Elapsed time"]/3600),vaclog["1mbar inj"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((vaclog["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 100mbar conductance gauge')

In [63]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [64]:

#print hv comments
print(pd.unique(vaclog["Live comments"]))
vaclog_comm = vaclog.loc[:,["Time","Live comments"]]
vaclog_comm.dropna(inplace=True)
print(vaclog_comm)
vaclog_comm.index

[nan 'closed turbo valve' 'turbo valve open, pressure rise too quick'
 'starting lhe transfer' 'closed turbo' 'injection complete'
 'turbo valve opened']
                   Time                              Live comments
68  2023-04-28 09:17:15                         closed turbo valve
77  2023-04-28 09:18:33  turbo valve open, pressure rise too quick
79  2023-04-28 09:18:50                      starting lhe transfer
218 2023-04-28 09:38:45                               closed turbo
236 2023-04-28 09:41:24                         injection complete
850 2023-04-28 11:10:06                         turbo valve opened


Int64Index([68, 77, 79, 218, 236, 850], dtype='int64')

### Plotting

In [65]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [66]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution during RGA calibration")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')  

#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comm.index:
    plt.text(vaclog_comm["Time"][i],0.8,str(vaclog_comm["Live comments"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/Baryons.png')
plt.show()

In [67]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution during RGA calibration")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()